In [ ]:
import pandas as pd
import numpy  as np
from math import sin, cos, pi, sqrt
import random
import time
import sys
sys.path.insert(1, '../rtsvg')
from rtsvg import *
rt = RACETrack()

In [ ]:
circles      = []
n_circles    = 50
w,     h     = 400, 400
r_min, r_max = 10,  20
for i in range(n_circles):
    circles.append((w*random.random(), h*random.random(), random.randint(r_min, r_max)))
svg = [f'<svg x="{0}" y="{0}" width="{w}" height="{h}" >']
for i in range(n_circles):
    cx, cy, r = circles[i]
    _co_ = rt.co_mgr.getColor(i)
    svg.append(f'<circle cx="{cx}" cy="{cy}" r="{r}" stroke="#000000" fill="{_co_}" />')
svg.append(f'<rect x="1" y="1" width="{w-3}" height="{h-3}" fill="none" stroke="#000000"/></svg>')
rt.svgObject(''.join(svg))

In [ ]:
min_inter_circle_d = 5

# Find the "middle" circle
s, placed = [], []
for i in range(n_circles):
    cx, cy, r = circles[i]
    placed.append((cx,cy,r)) # this will be the updated placement ... just initializing here
    s.append((cx, i))
s.sort()
s2 = []
m = int(len(s)/2)
for _tuple_ in s[m-1:m+2]:
    cx, cy, r = circles[_tuple_[1]]
    s2.append((cy, _tuple_[1]))
s2.sort()
middle_i = s2[1][1]

# Place the middle circle
mx, my, mr = circles[middle_i]
placed[middle_i] = circles[middle_i]
placed_set = set([middle_i])

# Sort all circles relative to the middle
to_place = []
cx_m, cy_m, r_m = circles[middle_i]
for i in range(n_circles):
    if i == middle_i:
        continue
    cx, cy, r = circles[i]
    d         = sqrt((cx-cx_m)**2 + (cy-cy_m)**2)
    to_place.append((d, i))
to_place.sort()

def overlapsWithPlaced(cx,cy,r):
    for j in placed_set:
        cx2, cy2, r2 = placed[j]
        d = sqrt((cx-cx2)**2 + (cy-cy2)**2)
        if (d-min_inter_circle_d) < (r+r2):
            return True
    return False

# Place the circles
svg_track = [f'<svg x="{0}" y="{0}" width="{w}" height="{h}" >']
svg_track.append(f'<svg x="0" y="0" width="{w}" height="{h}" > {"".join(svg)} <rect x="0" y="0" width="{w}" height="{h}" fill="#ffffff" fill-opacity="0.9" /></svg>')
svg_track.append(f'<circle cx="{mx}" cy="{my}" r="{mr}" stroke="{rt.co_mgr.getColor(middle_i)}" stroke-width="3.0" fill="none" />')
for k in range(len(to_place)):
    i = to_place[k][1]
    cx, cy, r = circles[i]
    uv = rt.unitVector(((mx,my),(cx,cy)))
    if uv[0] == 0 and uv[1] == 0:
        uv = (1,0)
    fail_after = 0
    while overlapsWithPlaced(cx,cy,r) and fail_after < 1000:
        cx, cy = cx + uv[0]*min_inter_circle_d, cy + uv[1]*min_inter_circle_d
        svg_track.append(f'<circle cx="{cx}" cy="{cy}" r="{2}" fill="{rt.co_mgr.getColor(i)}" stroke="none" />')
        fail_after += 1
    fail_after = 0
    last_cx, last_cy = cx, cy
    while overlapsWithPlaced(cx,cy,r) == False and fail_after < 1000:
        last_cx, last_cy = cx, cy
        cx, cy = cx - uv[0]*min_inter_circle_d/4, cy - uv[1]*min_inter_circle_d/4
        svg_track.append(f'<circle cx="{cx}" cy="{cy}" r="{5}" stroke="{rt.co_mgr.getColor(i)}" stroke-width="0.1" fill="none" />')
        fail_after += 1
    placed_set.add(i)
    placed[i] = (last_cx,last_cy,r)
svg_track.append(f'<rect x="1" y="1" width="{w-3}" height="{h-3}" fill="none" stroke="#000000"/></svg>')

svg_placed = [f'<svg x="{0}" y="{0}" width="{w}" height="{h}" >']
for i in range(n_circles):
    cx, cy, r = placed[i]
    _co_ = rt.co_mgr.getColor(i)
    svg_placed.append(f'<circle cx="{cx}" cy="{cy}" r="{r}" stroke="#000000" fill="{_co_}" />')
svg_placed.append(f'<rect x="1" y="1" width="{w-3}" height="{h-3}" fill="none" stroke="#000000"/></svg>')
rt.tile([''.join(svg), ''.join(svg_track), ''.join(svg_placed)])

In [ ]:
_placed_ = rt.crunchCircles(circles, min_d=min_inter_circle_d)
svg_placed_alg = [f'<svg x="{0}" y="{0}" width="{w}" height="{h}" >']
for i in range(n_circles):
    cx, cy, r = _placed_[i]
    _co_ = rt.co_mgr.getColor(i)
    svg_placed_alg.append(f'<circle cx="{cx}" cy="{cy}" r="{r}" stroke="#000000" fill="{_co_}" />')
svg_placed_alg.append(f'<rect x="1" y="1" width="{w-3}" height="{h-3}" fill="none" stroke="#000000"/></svg>')
rt.tile([''.join(svg), ''.join(svg_placed_alg)])